In [261]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [263]:
url = 'https://www.laalmanac.com/crime/cr03.php#:~:text=Other%20Los%20Angeles%20County%20cities,8%20crimes%20each%20per%201%2C000'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [265]:
table = soup.find('table')

In [267]:
column_data = table.find_all('tr')

In [269]:
list = []
for row in column_data[1:]:
    row_data = row.find_all('td')
    individual_row_data = [data.text.strip() for data in row_data]
    list.append(individual_row_data)

In [271]:
filtered_list = [item for item in list if item]

In [273]:
odd_index_elements = filtered_list[1::2]

In [275]:
even_index_elements = filtered_list[::2]

In [277]:
#odd_index_elements

In [279]:
#len(even_index_elements)

In [281]:
new_odd = odd_index_elements[:89]

In [283]:
new_even = even_index_elements[:89]

In [285]:
for sublist in new_odd:
    if len(sublist) > 1:  # Ensure there is a second element to pop
        sublist.pop(0)

In [287]:
for sublist in new_even:
    if len(sublist) > 1:  # Ensure there is a second element to pop
        sublist.pop(1)

In [289]:
df1 = pd.DataFrame(new_even, columns=['City', 'Total Crime #', 'Homicide #', 'Sexual Violence #', 'Robbery #', 'Aggrevated Assualt #'])

In [293]:
df2 = pd.DataFrame(new_odd, columns=['Total Crime %', 'Homicide %', 'Sexual Violence %', 'Robbery %', 'Aggrevated Assualt%'])

In [299]:
city_names = [sublist[0] for sublist in new_even if sublist]

In [301]:
df2.insert(0, 'City', city_names)


In [303]:
merged_df = pd.merge(df1, df2, on='City', how='inner')


In [305]:
merged_df

,City,Total Crime #,Homicide #,Sexual Violence #,Robbery #,Aggrevated Assualt #,Total Crime %,Homicide %,Sexual Violence %,Robbery %,Aggrevated Assualt%
0,County Total*,"61,193",683,"3,654","17,635","39,221",6.2,0.1,0.4,1.8,4.0
1,Agoura Hills,34,0,3,7,24,1.7,0.0,0.2,0.4,1.2
2,Alhambra,195,4,12,85,94,2.4,0.0,0.1,1.0,1.1
3,Arcadia,152,0,4,61,87,2.7,0.0,0.1,1.1,1.6
4,Artesia,82,2,4,15,61,5.1,0.1,0.2,0.9,3.8
...,...,...,...,...,...,...,...,...,...,...,...
84,Walnut,43,0,3,14,26,1.5,0.0,0.1,0.5,0.9
85,West Covina,357,1,25,128,203,3.3,0.0,0.2,1.2,1.9
86,West Hollywood,311,0,30,130,151,8.8,0.0,0.9,3.7,4.3
87,Westlake Village,18,0,1,5,12,2.3,0.0,0.1,0.6,1.5


In [307]:
merged_df['City'] = merged_df['City'].str.replace('*', '', regex=False)

In [309]:
def clean_and_convert(value):
    value = value.replace(',', '').rstrip('%').strip()  # Remove commas, trailing '+', and whitespace
    return pd.to_numeric(value, errors='coerce')  # Convert to numeric, setting errors to NaN

merged_df.loc[:, merged_df.columns != 'City'] = merged_df.loc[:, merged_df.columns != 'City'].applymap(clean_and_convert)

C:\Users\pminh\AppData\Local\Temp\ipykernel_16660\1353660357.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged_df.loc[:, merged_df.columns != 'City'] = merged_df.loc[:, merged_df.columns != 'City'].applymap(clean_and_convert)


In [311]:
merged_df

,City,Total Crime #,Homicide #,Sexual Violence #,Robbery #,Aggrevated Assualt #,Total Crime %,Homicide %,Sexual Violence %,Robbery %,Aggrevated Assualt%
0,County Total,61193,683,3654,17635,39221,6.2,0.1,0.4,1.8,4.0
1,Agoura Hills,34,0,3,7,24,1.7,0.0,0.2,0.4,1.2
2,Alhambra,195,4,12,85,94,2.4,0.0,0.1,1.0,1.1
3,Arcadia,152,0,4,61,87,2.7,0.0,0.1,1.1,1.6
4,Artesia,82,2,4,15,61,5.1,0.1,0.2,0.9,3.8
...,...,...,...,...,...,...,...,...,...,...,...
84,Walnut,43,0,3,14,26,1.5,0.0,0.1,0.5,0.9
85,West Covina,357,1,25,128,203,3.3,0.0,0.2,1.2,1.9
86,West Hollywood,311,0,30,130,151,8.8,0.0,0.9,3.7,4.3
87,Westlake Village,18,0,1,5,12,2.3,0.0,0.1,0.6,1.5


In [313]:
merged_df.to_csv('crime_rate.csv', index=False)